In [1]:
# ==============================================================================
# The Harmonizer Agent - All-in-One Notebook
#
# This single cell contains the entire application logic for the agent.
# It reads the .env file for the API key and the my_app_reqs.in for the
# problem to solve.
# ==============================================================================

# ==============================================================================
# 0. IMPORTS & SETUP
# ==============================================================================
import os
import sys
import re
import json
import heapq
import subprocess
import tempfile
from dataclasses import dataclass, field
from typing import Set, Tuple, FrozenSet, Any, Callable, List, Dict, Optional

# Third-party imports
try:
    from dotenv import load_dotenv
    from langchain_openai import ChatOpenAI
    from langchain_core.messages import SystemMessage, HumanMessage
except ImportError:
    print("Required libraries not found. Please run: pip install langchain-openai python-dotenv pip-tools openai")
    # This will stop the script if libs are not installed.
    sys.exit(1)


print("All libraries imported successfully.")

# ==============================================================================
# 1. CONFIGURATION (from harmonizer/config.py)
# ==============================================================================
# Load environment variables from a .env file
load_dotenv()

# --- LLM Provider Configuration ---
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_API_BASE = "https://openrouter.ai/api/v1"
MODEL_NAME = "deepseek/deepseek-chat"

# --- Recommended HTTP Headers for OpenRouter ---
APP_TITLE = "TheHarmonizerAgent-Notebook"
HTTP_REFERER = "http://localhost/harmonizer"

def validate_config():
    """Validates that essential configuration is set."""
    if not OPENROUTER_API_KEY or not OPENROUTER_API_KEY.startswith("sk-or-v1-"):
        raise ValueError(
            "CRITICAL ERROR: OPENROUTER_API_KEY is not defined correctly in your .env file. "
            "Please create a .env file with your valid OpenRouter key by running the setup cell."
        )
    print("✅ Configuration loaded successfully.")
    print(f"   Model: {MODEL_NAME}")
    print(f"   API Key (truncated): ...{OPENROUTER_API_KEY[-6:]}")

# ==============================================================================
# 2. STATE DEFINITIONS (from harmonizer/state.py)
# ==============================================================================
@dataclass(frozen=True)
class Requirement:
    """Represents a single package requirement, e.g., 'pandas==1.5.0'."""
    name: str
    specifier: str

    def __str__(self):
        return f"{self.name}{self.specifier}"

    @classmethod
    def from_string(cls, req_string: str):
        req_string = req_string.strip()
        match = re.match(r"([a-zA-Z0-9_-]+)\s*([~<>=!,\.\d\w]*)", req_string)
        if not match:
            raise ValueError(f"Could not parse requirement: {req_string}")
        name, specifier = match.groups()
        return cls(name=name.lower(), specifier=specifier)

@dataclass(frozen=True)
class Action:
    """Represents a single modification to the set of requirements."""
    description: str
    cost: float

@dataclass(frozen=True)
class ProblemState:
    """Represents a state in our search space."""
    requirements: FrozenSet[Requirement]

    def __str__(self):
        if not self.requirements:
            return "# No requirements"
        return "\n".join(sorted(str(req) for req in self.requirements))
        
    @classmethod
    def from_string(cls, text: str):
        reqs = frozenset(
            Requirement.from_string(line)
            for line in text.splitlines()
            if line.strip() and not line.strip().startswith("#")
        )
        return cls(requirements=reqs)

# ==============================================================================
# 3. ENVIRONMENT (from harmonizer/environment.py)
# ==============================================================================
@dataclass
class PipCompileResult:
    """Stores the outcome of a pip-compile execution."""
    success: bool
    output: str
    error: str

class PipCompileEnvironment:
    """Manages a temporary environment to run pip-compile safely."""

    def run(self, state: ProblemState) -> PipCompileResult:
        """Runs pip-compile on a given set of requirements."""
        with tempfile.TemporaryDirectory() as temp_dir:
            input_file_path = os.path.join(temp_dir, "requirements.in")
            output_file_path = os.path.join(temp_dir, "requirements.txt")

            with open(input_file_path, "w", encoding='utf-8') as f:
                f.write(str(state))

            command = ["pip-compile", "--output-file", output_file_path, input_file_path, "--quiet"]
            process = subprocess.run(command, capture_output=True, text=True, encoding='utf-8')

            if process.returncode == 0:
                with open(output_file_path, "r", encoding='utf-8') as f:
                    resolved_reqs = f.read()
                return PipCompileResult(success=True, output=resolved_reqs, error="")
            else:
                return PipCompileResult(success=False, output="", error=process.stderr)

# ==============================================================================
# 4. LLM REASONING ENGINE (from harmonizer/llm_reasoning.py)
# ==============================================================================
class LLMReasoningEngine:
    """Uses an LLM to reason about dependency conflicts and suggest solutions."""
    def __init__(self):
        self.llm = ChatOpenAI(
            model_name=MODEL_NAME,
            openai_api_key=OPENROUTER_API_KEY,
            openai_api_base=OPENROUTER_API_BASE,
            temperature=0.1,
            max_tokens=1024,
            default_headers={"X-Title": APP_TITLE, "HTTP-Referer": HTTP_REFERER}
        )
        self._system_prompt = self._create_system_prompt()

    def _create_system_prompt(self) -> str:
        return """
You are "The Harmonizer," an expert Python dependency resolution assistant.
Your goal is to fix a `requirements.in` file so that `pip-compile` can successfully resolve it.
You will be given the current (failing) `requirements.in` and the error message from `pip-compile`.
You must analyze the error and propose a list of concrete, surgical modifications to the requirements.

You MUST respond with a JSON object containing a list called "suggestions".
Each suggestion in the list must be a JSON object with three keys:
1. "action_type": A string, either "MODIFY" or "REMOVE".
2. "package_name": The name of the package to change (e.g., "pandas").
3. "new_specifier": The new version specifier (e.g., "<2.0,>=1.5" or "==1.5.3"). This key is ONLY for the "MODIFY" action.

- Prioritize the least disruptive changes first. Modifying a specifier is better than removing a package.
- Be precise. If the error says `pandas<2.0` is needed, a good suggestion is `pandas<2.0,>=1.0` or a specific compatible version like `1.5.3`.
- Do not suggest adding new packages, only modify or remove existing ones.
- Think step-by-step to find the root cause. The error log is your most important clue.

Example Response:
{
  "suggestions": [
    {
      "action_type": "MODIFY",
      "package_name": "pandas",
      "new_specifier": "<2.0"
    },
    {
      "action_type": "REMOVE",
      "package_name": "problematic-package"
    }
  ]
}
"""

    def suggest_modifications(self, current_state: ProblemState, error_log: str) -> list[tuple[ProblemState, Action]]:
        """Asks the LLM for suggestions to fix the dependency conflict."""
        human_prompt = f"""
Here is the current `requirements.in` file:
---
{str(current_state)}
---

Here is the error message from `pip-compile`:
---
{error_log}
---

Please provide your suggestions in the specified JSON format.
"""
        messages = [SystemMessage(content=self._system_prompt), HumanMessage(content=human_prompt)]
        response_content = ""
        try:
            response = self.llm.invoke(messages)
            response_content = response.content
            # The LLM sometimes wraps the JSON in ```json ... ```, so we extract it.
            json_match = re.search(r'```json\s*([\s\S]*?)\s*```', response_content)
            if json_match:
                response_content = json_match.group(1)
            
            suggestions_json = json.loads(response_content)
            return self._parse_llm_suggestions(current_state, suggestions_json)
        except (json.JSONDecodeError, KeyError, Exception) as e:
            print(f"⚠️  LLM response was not valid JSON or was malformed. Error: {e}")
            print(f"   Raw response: {response_content}")
            return []

    def _parse_llm_suggestions(self, base_state: ProblemState, suggestions_json: dict) -> list[tuple[ProblemState, Action]]:
        """Parses the JSON from the LLM into new states and actions."""
        new_states = []
        base_reqs_dict = {req.name: req for req in base_state.requirements}

        for suggestion in suggestions_json.get("suggestions", []):
            try:
                action_type = suggestion["action_type"]
                package_name = suggestion["package_name"].lower()
                if package_name not in base_reqs_dict: continue

                new_reqs = dict(base_reqs_dict)
                action = None

                if action_type == "MODIFY":
                    new_specifier = suggestion["new_specifier"]
                    original_req = base_reqs_dict[package_name]
                    new_reqs[package_name] = Requirement(name=package_name, specifier=new_specifier)
                    action = Action(
                        description=f"MODIFIED '{package_name}' from '{original_req.specifier}' to '{new_specifier}'",
                        cost=1.0 if "==" in new_specifier else 1.5
                    )
                elif action_type == "REMOVE":
                    del new_reqs[package_name]
                    action = Action(description=f"REMOVED '{package_name}'", cost=10.0)

                if action:
                    new_state = ProblemState(requirements=frozenset(new_reqs.values()))
                    new_states.append((new_state, action))
            except (KeyError, TypeError) as e:
                print(f"⚠️  Skipping malformed suggestion: {suggestion}. Error: {e}")
                continue
        return new_states

# ==============================================================================
# 5. A* SEARCH ALGORITHM (from harmonizer/search.py)
# ==============================================================================
@dataclass(order=True)
class AStarNode:
    """A node in the A* search tree."""
    priority: float = field(init=False)
    state: ProblemState = field(compare=False)
    g_score: float = field(compare=False)
    h_score: float = field(compare=False)
    parent: Optional['AStarNode'] = field(default=None, compare=False)
    action: Optional[Action] = field(default=None, compare=False)

    def __post_init__(self):
        self.priority = self.g_score + self.h_score

def reconstruct_path(node: AStarNode) -> list[tuple[ProblemState, Optional[Action]]]:
    """Builds the solution path by backtracking from the goal node."""
    path = []
    while node:
        path.append((node.state, node.action))
        node = node.parent
    return path[::-1]

def a_star_search(
    initial_state: ProblemState,
    get_neighbors: Callable[[ProblemState], List[Tuple[ProblemState, Action]]],
    is_goal: Callable[[ProblemState], bool],
    heuristic: Callable[[ProblemState], float],
    max_iterations: int = 50
) -> Optional[list[tuple[ProblemState, Optional[Action]]]]:
    """Performs an A* search."""
    start_node = AStarNode(state=initial_state, g_score=0, h_score=heuristic(initial_state))
    open_set: List[AStarNode] = [start_node]
    processed_g_scores: Dict[ProblemState, float] = {}
    iterations = 0

    while open_set and iterations < max_iterations:
        iterations += 1
        current_node = heapq.heappop(open_set)

        if is_goal(current_node.state):
            print(f"\n✅ Solution Found in {iterations} steps!")
            return reconstruct_path(current_node)

        if current_node.state in processed_g_scores and current_node.g_score >= processed_g_scores[current_node.state]:
            continue
        
        processed_g_scores[current_node.state] = current_node.g_score
        
        print(f"\n[Step {iterations}] Exploring state (g={current_node.g_score:.1f}, h={current_node.h_score:.1f})...")
        print(f"  Action: {current_node.action.description if current_node.action else 'Initial state'}")
        
        neighbors = get_neighbors(current_node.state)
        print(f"  LLM suggested {len(neighbors)} potential fixes.")

        for neighbor_state, action in neighbors:
            tentative_g_score = current_node.g_score + action.cost
            if neighbor_state in processed_g_scores and tentative_g_score >= processed_g_scores[neighbor_state]:
                continue
            neighbor_node = AStarNode(
                state=neighbor_state, g_score=tentative_g_score, h_score=heuristic(neighbor_state),
                parent=current_node, action=action
            )
            heapq.heappush(open_set, neighbor_node)
            
    print(f"\n❌ Search failed after {iterations} iterations. No solution found.")
    return None

# ==============================================================================
# 6. HARMONIZER AGENT (from harmonizer/agent.py)
# ==============================================================================
class HarmonizerAgent:
    """An intelligent agent that resolves Python dependency conflicts."""
    def __init__(self):
        self.env = PipCompileEnvironment()
        self.llm_engine = LLMReasoningEngine()
        self.memoized_checks: dict[ProblemState, PipCompileResult] = {}

    def _check_state(self, state: ProblemState) -> PipCompileResult:
        """Checks if a state is resolvable, using a cache."""
        if state not in self.memoized_checks:
            print(f"  -> Running pip-compile on state...")
            self.memoized_checks[state] = self.env.run(state)
        return self.memoized_checks[state]

    def _is_goal(self, state: ProblemState) -> bool:
        return self._check_state(state).success

    def _get_neighbors(self, state: ProblemState) -> List[Tuple[ProblemState, Action]]:
        result = self._check_state(state)
        if result.success: return []
        print("  -> Conflict detected. Asking LLM for suggestions...")
        return self.llm_engine.suggest_modifications(state, result.error)

    def _heuristic(self, state: ProblemState) -> float:
        # A simple heuristic. A better one could be based on error complexity.
        return 1.0

    def solve(self, initial_requirements: str) -> Optional[list[tuple[ProblemState, Optional[Action]]]]:
        """Attempts to find a resolvable set of requirements."""
        try:
            initial_state = ProblemState.from_string(initial_requirements)
        except ValueError as e:
            print(f"Error parsing initial requirements: {e}")
            return None

        print("--- Starting Harmonizer Agent ---")
        print("Initial requirements:")
        print(str(initial_state))
        print("---------------------------------")
        
        if self._check_state(initial_state).success:
            print("✅ Initial requirements are already valid!")
            return [(initial_state, None)]

        return a_star_search(
            initial_state=initial_state, get_neighbors=self._get_neighbors,
            is_goal=self._is_goal, heuristic=self._heuristic,
        )

# ==============================================================================
# 7. MAIN EXECUTION LOGIC (from main.py)
# ==============================================================================
def run_harmonizer():
    # In a notebook, we define the input file directly.
    requirements_file = "requirements.in"
    
    print(f"--- Running Harmonizer on '{requirements_file}' ---")
    
    try:
        validate_config()
    except ValueError as e:
        print(str(e))
        return

    try:
        with open(requirements_file, "r") as f:
            initial_reqs = f.read()
    except FileNotFoundError:
        print(f"Error: The file '{requirements_file}' was not found.")
        print("Please run the setup cells at the top of the notebook to create it.")
        return

    agent = HarmonizerAgent()
    solution_path = agent.solve(initial_reqs)

    if solution_path:
        final_state, _ = solution_path[-1]
        total_cost = sum(action.cost for _, action in solution_path if action)
        
        print("\n============================================================")
        print("=== Final Resolved requirements.in ===")
        print("============================================================")
        print(str(final_state))
        print("\n============================================================")
        print(f"=== Path to Solution (Total Cost: {total_cost:.1f}) ===")
        print("============================================================")
        for i, (state, action) in enumerate(solution_path, 1):
            if action:
                print(f"{i}. [Cost: {action.cost:.1f}] {action.description}")
            else:
                print("1. Start with initial requirements.")
    else:
        print("\n============================================================")
        print("=== No solution could be found within the search limits. ===")
        print("============================================================")

# --- Execute the agent ---
run_harmonizer()

Python-dotenv could not parse statement starting at line 1


All libraries imported successfully.
--- Running Harmonizer on 'requirements.in' ---
✅ Configuration loaded successfully.
   Model: deepseek/deepseek-chat
   API Key (truncated): ...b81702
--- Starting Harmonizer Agent ---
Initial requirements:
pandas==2.1.0
scikit-learn==1.2.0
---------------------------------
  -> Running pip-compile on state...
✅ Initial requirements are already valid!

=== Final Resolved requirements.in ===
pandas==2.1.0
scikit-learn==1.2.0

=== Path to Solution (Total Cost: 0.0) ===
1. Start with initial requirements.
